In [ ]:
# run IPYNB that imports all the relevant packages and functions
%run ../ms_packages_and_functions.ipynb

In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir   = pjoin(os.getcwd(), 'raw_data')


In [ ]:
data = loadPickle('processed_data/rj1031209_CenterOut_data_10msBin_with500buffer_MIonly.pkl')

In [ ]:
rand_state = '003' #np.random.randint(1000000000)

In [ ]:
folder_path = pjoin(output_dir,'RJ_FN+Embedding_fullTrial-{}'.format(rand_state))

if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    print("created directory: {}".format(folder_path))
else:
    print("All files will be saved in: {}".format(folder_path))


In [ ]:
fig_dir    = pjoin(folder_path,'figures')
if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)
    print("created directory: {}".format(fig_dir))
else:
    print("All figures will be saved in: {}".format(fig_dir))

# 1. Generate Full Trial Data FNs

In [ ]:
save_FNs = True
if 'data_FNs' not in locals():
    print("data_FNs not found")
    print("creating data_FNs")

    data_FNs = []

    direction_FNs = []

    for direction in range(0,len(data)):

        direction_FNs.append([])
        
        for trial in range(0,len(data[direction]['trialData'])):
            
            status("> Generating FN for direction: {}/{},trial: {}/{}".format(direction, len(data), \
                    trial,len(data[direction]['trialData'])))
            trialData = data[direction]['trialData'][trial]['trialBinnedSpikes']
            tempFN = make_FN(trialData,metric='fMI',plot=False)
            direction_FNs[direction].append(tempFN)

    data_FNs.append(direction_FNs)
    if save_FNs:
        save_dir = pjoin(folder_path,'RJ_CenterOut_FN_10ms_fullTrial.pkl')
        savePickle(save_dir,data_FNs)

# 2. Flatten FNs

In [ ]:
# load data
data_FNs= loadPickle('processed_data/RJ_FN+Embedding_fullTrial-001/RJ_CenterOut_FN_10ms_fullTrial.pkl')

In [ ]:
trials = loadPickle('processed_data/rj031209_trials_by_cutoff.pkl')
cutoff = 1.5

In [ ]:
dirs = np.arange(8)
FN_flattened = {'data':[flattenNetwork(W) for direction,x in enumerate(data_FNs[0]) for trial,W in enumerate(x) if trial in trials[cutoff]['trials'][direction] and direction in dirs],
                'FN_info': [[direction,trial] for direction in range(len(data_FNs[0])) 
                                              for trial in range(len(data_FNs[0][direction])) if trial in trials[cutoff]['trials'][direction] and direction in dirs]}

savePickle(pjoin(folder_path,f'RJ_CenterOut_FN_10ms_fullTrial_flattened-{cutoff}.pkl'),FN_flattened)

In [ ]:
dirs

# 3. Embed

In [ ]:
print(">> Embedding data...")
rand_state = 82437048#np.random.randint(1000000000)
umap_reducer = umap.UMAP(n_neighbors=100, n_components=2,min_dist=0.5,random_state=rand_state,metric='cosine')
embeddings_2d = umap_reducer.fit_transform(FN_flattened['data'])
print("done embedding!")
save_dir = pjoin(folder_path,f'RJ_FN+Embedding_fullTrial_UMAP_allDirs_{rand_state}.pkl')
savePickle(save_dir,embeddings_2d)

In [ ]:
save_dir

In [ ]:
figure = plt.figure(figsize=(20,20))
cmap = sns.color_palette("husl",8)

ax = figure.add_subplot(111)
for s in range(0,len(data)):
    idx = [i for i,x in enumerate(FN_flattened['FN_info']) if x[0]==s]
    colors = [cmap[x[0]] for x in np.array(FN_flattened['FN_info'])[idx]]
    ax.scatter(embeddings_2d[idx, 0], embeddings_2d[idx, 1],c=colors,marker='.',alpha=0.2)
    sns.kdeplot(x=embeddings_2d[idx, 0], y=embeddings_2d[idx, 1],shade= True,levels=2, color=cmap[s],thresh=0.1,alpha=0.2)
    ax.axis('equal')
#
colors =  [cmap[x[0]] for x in FN_flattened['FN_info']]
ax.scatter(embeddings_2d[:,0], embeddings_2d[:,1], c=colors,s= 100, alpha=1) #edgecolor = (0,0,0)

ax.set_xlabel('UMAP X (au)')
ax.set_ylabel('UMAP Y (au)')
ax.set_title('RS_UMAP_fullTrial')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.savefig(pjoin(fig_dir,f'RS_UMAP_allDirs_{rand_state}.png'))

In [ ]:
min_dist_vals = [0,0.01,0.05,0.1,0.5,1]
n_neighbor_vals = [5,15,30,50,100]
cmap = sns.color_palette('husl',8)
colors = [cmap[x[0]] for x in np.array(FN_flattened['FN_info'])]

figure = plt.figure(figsize=(20,20))
plot_num = 0
for row, min_dist in enumerate(min_dist_vals):
    for col,n_neighbor in enumerate(n_neighbor_vals):
        umap_reducer = umap.UMAP(n_neighbors=n_neighbor, n_components=2,min_dist=min_dist,random_state=rand_state,metric='cosine')
        embeddings_2d = umap_reducer.fit_transform(FN_flattened['data'])
        plot_num +=1
        plt.subplot(len(min_dist_vals),len(n_neighbor_vals),plot_num)
        plt.title("m-dist: {},nn ={}".format(min_dist,n_neighbor))
        plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1],c=colors,marker='.',alpha=1)
        
plt.savefig(pjoin(fig_dir,f'RJ_UMAP-params-grid_allDirs_{rand_state}.png'))
        

# 6. GAS

In [ ]:
trials = loadPickle('processed_data/rj031209_trials_by_cutoff.pkl')

In [ ]:
data_FNs = loadPickle('processed_data/RJ_FN+Embedding_fullTrial-001/RJ_CenterOut_FN_10ms_fullTrial.pkl')

In [ ]:
# resample data FNs to only include trials in cutoff
cutoff = 1.5
data_alignment_scores = []

all_trials = [[dir,trial] for dir in range(0,len(data_FNs[0]))
                        for trial in range(0,len(data_FNs[0][dir]))]
                        
FN_pairs = [y for y in itertools.combinations(all_trials,2)]

cdist = [circdistance(8,y[0][0],y[1][0]) for y in FN_pairs]

[data_alignment_scores.append([]) for x in range(8)]#np.unique(cdist)]

for c, pair in zip(cdist,FN_pairs):
    if (pair[0][1] in trials[cutoff]['trials'][pair[0][0]]) and (pair[1][1] in trials[cutoff]['trials'][pair[1][0]]):
        score = align_score(data_FNs[0][pair[0][0]][pair[0][1]],data_FNs[0][pair[1][0]][pair[1][1]])
        data_alignment_scores[c].append(score)

save_dir = pjoin(folder_path,'data_alignment_score.pkl')
savePickle(save_dir,data_alignment_scores)

In [ ]:
null_FNs = loadPickle('processed_data/RJ_nulls-poisson-full-000/poisson_FNs_fullTrial_100.pkl')

In [ ]:
null_alignment_scores = []

for i,FNs in enumerate(null_FNs):
    status('getting alignment scores for nulls: sim#{}'.format(i))
    null_alignment_scores.append([])

    all_trials = [[dir,trial] for dir in range(0,len(FNs[0]))
                            for trial in range(0,len(FNs[0][dir]))]
    FN_pairs = [y for y in itertools.combinations(all_trials,2)]

    cdist = [circdistance(8,y[0][0],y[1][0]) for y in FN_pairs]

    [null_alignment_scores[i].append([]) for x in  range(8)]#np.unique(cdist)]

    for c, pair in zip(cdist,FN_pairs):
        if (pair[0][1] in trials[cutoff]['trials'][pair[0][0]]) and (pair[1][1] in trials[cutoff]['trials'][pair[1][0]]):
            score = align_score(FNs[0][pair[0][0]][pair[0][1]],FNs[0][pair[1][0]][pair[1][1]])
            null_alignment_scores[i][c].append(score)
save_dir = pjoin(folder_path,'null_alignment_score.pkl')
savePickle(save_dir,null_alignment_scores)